## Import Dependencies

In [1]:
#data preprocessing
import pandas as pd
#produces a prediction model in the form of an ensemble of weak prediction models, typically decision tree
import xgboost as xgb
#the outcome (dependent variable) has only a limited number of possible values. 
#Logistic Regression is used when response variable is categorical in nature.
from sklearn.linear_model import LogisticRegression
#A random forest is a meta estimator that fits a number of decision tree classifiers 
#on various sub-samples of the dataset and use averaging to improve the predictive 
#accuracy and control over-fitting.
from sklearn.ensemble import RandomForestClassifier
#a discriminative classifier formally defined by a separating hyperplane.
from sklearn.svm import SVC
#displayd data
from IPython.display import display


#%matplotlib inline

/home/angloher/projects/Predicting_Winning_Teams/PredictEnv/local/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
import os
# Read data and drop redundant column.
loc = "/home/angloher/projects/Predicting_Winning_Teams/Datasets" #('Scotish_league',4)
leagues=[('Premier_league',5),
        ('Bundesliga',2),('French_league',2),('Portoguese_league',1),('Turkish_league',1),
         ('Greek_league',1),('Belgian_league',1),
         ('La_Liga',2),('Eredivisie',1),('SerieA',2)]
         #('Swiss_league',1),('Finish_league',1),('Austrian_league',1),
         #('Scotish_league',4),('Chinese_league',1),('Norwegian_league',1),
         #('Russian_league',1),('Swedish_league',1),('Brazilian_league',1),
         #('Polish_league',1),('Romanian_league',1),('Mexican_league',1),
         #('Japanese_league',1),('Danish_league',1)]#]
train_val_split_date=['07','13','2018']#month/day/year
#train_val_stop_date=['02','20','2018']
files=[]
betting_files=[]
num_last_games=20
num_last_seasons=6

for league,divisions in leagues:
    for i in range(1,divisions+1):
        #print len(pd.read_csv(os.path.join(loc,league,str(i),'Results','final_dataset.csv')))
        #print len(pd.read_csv(os.path.join(loc,league,str(i),'Results','betting_odds.csv')))
        files.append(pd.read_csv(os.path.join(loc,league,str(i),'Results','final_dataset.csv')))
        betting_files.append(pd.read_csv(os.path.join(loc,league,str(i),'Results','betting_odds.csv'))[['B365H','B365D','B365A']])
data=pd.concat(files)
betting_data=pd.concat(betting_files)

print len(data),len(betting_data)
data=pd.concat([data,betting_data],axis=1)

display(data.head())
#Full Time Result (H=Home Win, D=Draw, A=Away Win)
#HTGD - Home team goal difference
#ATGD - away team goal difference
#HTP - Home team points
#ATP - Away team points
#DiffFormPts Diff in points
#DiffLP - Differnece in last years prediction

#Input - 12 other features (fouls, shots, goals, misses,corners, red card, yellow cards)
#Output - Full Time Result (H=Home Win, D=Draw, A=Away Win) 

161940 161940


,Unnamed: 0,AC,ACC,ACS,AM10Conceded,AM10OppConceded,AM10OppPoints,AM10OppScored,AM10Scored,AM11Conceded,...,LY4HSS,LY4HSTC,LY4HSTS,LY4HTGC,LY4HTGS,LY4HTP,MW,B365H,B365D,B365A
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN
4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN


## Data Exploration

In [3]:
#what is the win rate for the home team?

# Total number of matches.
n_matches = data.shape[0]

# Calculate number of features. -1 because we are saving one as the target variable (win/lose/draw)
n_features = data.shape[1] - 1

# Calculate matches won by home team.
n_homewins = len(data[data.FTR == 'H'])

# Calculate win rate for home team.
win_rate = (float(n_homewins) / (n_matches)) * 100

# Print the results
print "Total number of matches: {}".format(n_matches)
print "Number of features: {}".format(n_features)
print "Number of matches won by home team: {}".format(n_homewins)
print "Win rate of home team: {:.2f}%".format(win_rate)

Total number of matches: 161940
Number of features: 357
Number of matches won by home team: 74593
Win rate of home team: 46.06%


## Preparing the Data

In [4]:
# Separate into feature set and target variable
#FTR = Full Time Result (H=Home Win, D=Draw, A=Away Win)
train_set=data[pd.to_datetime(data.Date)<pd.to_datetime('{}-{}-{}'.format(train_val_split_date[0],train_val_split_date[1],train_val_split_date[2]))]
validate_new_result_set=data[pd.to_datetime(data.Date)>pd.to_datetime('{}-{}-{}'.format(train_val_split_date[0],train_val_split_date[1],train_val_split_date[2]))]
#validate_new_result_set=temp[pd.to_datetime(temp.Date)<pd.to_datetime('{}-{}-{}'.format(train_val_stop_date[0],train_val_stop_date[1],train_val_stop_date[2]))]

print len(validate_new_result_set)
#train_set=data
#print list(train_set.columns)



# Standardising the data.
from sklearn.preprocessing import StandardScaler
import math
import numpy as np


#Center to the mean and component wise scale to unit variance.
cols = ['HTGS','HTGC','ATGS','ATGC','HTP','ATP','MW']#,
            #'HSS', 'ASS', 
            #'HSC', 'ASC', 'HSTS', 'ASTS', 'HSTC','ASTC', 
            #'HCS', 'ACS', 'HCC', 'ACC'] 

for i in range(1,num_last_games+1):
    cols.append('HM' + str(i)+'Scored')               
    cols.append('AM' + str(i)+'Scored')
    cols.append('HM' + str(i)+'Conceded')               
    cols.append('AM' + str(i)+'Conceded')
    cols.append('HM' + str(i)+'OppPoints')               
    cols.append('AM' + str(i)+'OppPoints')
    cols.append('HM' + str(i)+'OppScored')               
    cols.append('AM' + str(i)+'OppScored')
    cols.append('HM' + str(i)+'OppConceded')               
    cols.append('AM' + str(i)+'OppConceded')

for i in range(0,num_last_seasons-1):
    #cols.append('LY' + str(i)+'ACC')               
    #cols.append('LY' + str(i)+'HCC') 
    #cols.append('LY' + str(i)+'ACS')               
    #cols.append('LY' + str(i)+'HCS') 
    #cols.append('LY' + str(i)+'ASC')               
    #cols.append('LY' + str(i)+'HSC') 
    #cols.append('LY' + str(i)+'ASS')               
    #cols.append('LY' + str(i)+'HSS') 
    #cols.append('LY' + str(i)+'ASTC')               
    #cols.append('LY' + str(i)+'HSTC') 
    #cols.append('LY' + str(i)+'ASTS')               
    #cols.append('LY' + str(i)+'HSTS')
    cols.append('LY' + str(i)+'ATGC')               
    cols.append('LY' + str(i)+'HTGC') 
    cols.append('LY' + str(i)+'ATGS')               
    cols.append('LY' + str(i)+'HTGS') 
    cols.append('LY' + str(i)+'ATP')               
    cols.append('LY' + str(i)+'HTP') 
    cols.append('LY' + str(i)+'DivAT')               
    cols.append('LY' + str(i)+'DivHT')
    
for i in range(1,num_last_seasons*2-1):
    cols.append('LMUP' + str(i)+'HT')               
    cols.append('LMUP' + str(i)+'AT')    


    
#for col in cols:
#    train_set = train_set[train_set[col].notnull()]


X_all = train_set.drop(['FTR'],1)
X_validate_new_result_set=validate_new_result_set.drop(['FTR'],1)
y_all = train_set['FTR']
y_validate_new_result_set=validate_new_result_set['FTR']

scaler=StandardScaler()

#for col in cols:
    #print X_all.ix[~X_all[col].isnull()[col]]
   # print len(X_all[~X_all[col].isnull()][col].as_matrix().reshape(-1,1))
   # X_all.ix[~X_all[col].isnull(),col]= scaler.fit_transform(X_all[~X_all[col].isnull()][col].as_matrix().reshape(-1,1))
   # X_validate_new_result_set.ix[~X_validate_new_result_set[col].isnull(),col]= scaler.fit_transform(X_validate_new_result_set[~X_validate_new_result_set[col].isnull()][col].as_matrix().reshape(-1,1))
  
    #print X_all[~X_all[col].isnull()][col]
    
betting_cols=['B365H','B365D','B365A']
column=train_set['B365H']
column.fillna(1.888,inplace=True)#2.3601
train_set['B365H']=column
cols.append('B365H')
  
column=train_set['B365D']
column.fillna(1.888,inplace=True)#3.4961
train_set['B365D']=column
cols.append('B365D')

column=train_set['B365A']
column.fillna(1.888,inplace=True)#4.1027
train_set['B365A']=column
cols.append('B365A')
    
betting_cols=['B365H','B365D','B365A','Date','Div','HomeTeam','AwayTeam','MW']

for col in betting_cols:
    y_all=pd.concat([y_all,train_set[col]],axis=1)
    y_validate_new_result_set=pd.concat([y_validate_new_result_set,validate_new_result_set[col]],axis=1)
    
print len(X_all),len(y_all)    
print list(X_all.columns),len(X_all.columns)
    

1330


/home/angloher/projects/Predicting_Winning_Teams/PredictEnv/local/lib/python2.7/site-packages/pandas/core/generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/home/angloher/projects/Predicting_Winning_Teams/PredictEnv/lib/python2.7/site-packages/ipykernel_launcher.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/angloher/projects/Predicting_Winning_Teams/PredictEnv/lib/python2.7/site-packages/ipykernel_launcher.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

160610 160610
['Unnamed: 0', 'AC', 'ACC', 'ACS', 'AM10Conceded', 'AM10OppConceded', 'AM10OppPoints', 'AM10OppScored', 'AM10Scored', 'AM11Conceded', 'AM11OppConceded', 'AM11OppPoints', 'AM11OppScored', 'AM11Scored', 'AM12Conceded', 'AM12OppConceded', 'AM12OppPoints', 'AM12OppScored', 'AM12Scored', 'AM13Conceded', 'AM13OppConceded', 'AM13OppPoints', 'AM13OppScored', 'AM13Scored', 'AM14Conceded', 'AM14OppConceded', 'AM14OppPoints', 'AM14OppScored', 'AM14Scored', 'AM15Conceded', 'AM15OppConceded', 'AM15OppPoints', 'AM15OppScored', 'AM15Scored', 'AM16Conceded', 'AM16OppConceded', 'AM16OppPoints', 'AM16OppScored', 'AM16Scored', 'AM17Conceded', 'AM17OppConceded', 'AM17OppPoints', 'AM17OppScored', 'AM17Scored', 'AM18Conceded', 'AM18OppConceded', 'AM18OppPoints', 'AM18OppScored', 'AM18Scored', 'AM19Conceded', 'AM19OppConceded', 'AM19OppPoints', 'AM19OppScored', 'AM19Scored', 'AM1Conceded', 'AM1OppConceded', 'AM1OppPoints', 'AM1OppScored', 'AM1Scored', 'AM20Conceded', 'AM20OppConceded', 'AM20Opp

In [5]:
import time
import numpy as np
#last 5 games for both sides

def test_letter(c):
    if not c.isdigit():
        return 4
    else:
        return c
    
def preprocess_features(X,cols):
    ''' Preprocesses the football data and converts catagorical variables into dummy variables. '''
    Divisions=['E0','E1','E2','E3','EC','D1','D2','F1','F2',
               'SP1','SP2','B1','N1','G1','T1','P1','I1','I2']#,
               #'Br1','Swe1','Ru1','No1','Ch1','SC0','SC1','SC2','SC3',
               #'Au1','Fin1','Swi1','Den1','Jap1','Mex1','Po1','Ro1']
    cols.append('league')
    X['league']=None
    for div in Divisions:
        X[div[0:-1]]=0
        if div[0:-1] not in cols:
            cols.append(div[0:-1])
    for div in Divisions:
        X.ix[X['Div']==div,div[0:-1]]=1
        X.ix[X['Div']==div,'league']=float(test_letter(div[-1]))
    
    return X[cols]

val_cols=list(cols)
X_all=preprocess_features(X_all.copy(),val_cols)
val_cols=list(cols)
X_validate_new_result_set=preprocess_features(X_validate_new_result_set.copy(),val_cols)
#X_all=X_all[cols]

X_all.fillna(0,inplace=True)
X_validate_new_result_set.fillna(0,inplace=True)

print y_all[y_all['FTR'].isnull()==True]



print len(X_all)
print list(X_all.columns),len(X_all.columns)

print X_all.columns.to_series().groupby(X_all.dtypes).groups
#start=time.time()


/home/angloher/projects/Predicting_Winning_Teams/PredictEnv/lib/python2.7/site-packages/ipykernel_launcher.py:24: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


Empty DataFrame
Columns: [FTR, B365H, B365D, B365A, Date, Div, HomeTeam, AwayTeam, MW]
Index: []
160610
['HTGS', 'HTGC', 'ATGS', 'ATGC', 'HTP', 'ATP', 'MW', 'HM1Scored', 'AM1Scored', 'HM1Conceded', 'AM1Conceded', 'HM1OppPoints', 'AM1OppPoints', 'HM1OppScored', 'AM1OppScored', 'HM1OppConceded', 'AM1OppConceded', 'HM2Scored', 'AM2Scored', 'HM2Conceded', 'AM2Conceded', 'HM2OppPoints', 'AM2OppPoints', 'HM2OppScored', 'AM2OppScored', 'HM2OppConceded', 'AM2OppConceded', 'HM3Scored', 'AM3Scored', 'HM3Conceded', 'AM3Conceded', 'HM3OppPoints', 'AM3OppPoints', 'HM3OppScored', 'AM3OppScored', 'HM3OppConceded', 'AM3OppConceded', 'HM4Scored', 'AM4Scored', 'HM4Conceded', 'AM4Conceded', 'HM4OppPoints', 'AM4OppPoints', 'HM4OppScored', 'AM4OppScored', 'HM4OppConceded', 'AM4OppConceded', 'HM5Scored', 'AM5Scored', 'HM5Conceded', 'AM5Conceded', 'HM5OppPoints', 'AM5OppPoints', 'HM5OppScored', 'AM5OppScored', 'HM5OppConceded', 'AM5OppConceded', 'HM6Scored', 'AM6Scored', 'HM6Conceded', 'AM6Conceded', 'HM6Opp

In [ ]:
# Show the feature information by printing the first five rows

print "\nFeature values:"
display(X_all)

In [6]:
from sklearn.cross_validation import train_test_split
#fill leagueName data


# Shuffle and split the dataset into training and testing set.
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, 
                                                    test_size = len(y_all)/5,
                                                    random_state = 2,
                                                    stratify = y_all['FTR'])

X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, 
                                                    test_size = len(y_test)/2,
                                                    random_state = 2,
                                                    stratify = y_test['FTR'])


print 'length training data: {}\nlength test data: {}\nlength validation data: {}'.format(len(X_train),len(X_test),len(X_val))
print 'length new test data {}'.format(len(X_validate_new_result_set))

length training data: 128488
length test data: 16061
length validation data: 16061
length new test data 1330


In [7]:

def split_train_target(y_train):
    matrix=y_train
    i=0
    #print matrix
    for train in y_train:
        #print train[0]
        if train[0]=='H':
            matrix[i][0]=0
        elif train[0]=='D':
            matrix[i][0]=1
        else:
            matrix[i][0]=2
        i+=1
    matrix=np.array(matrix,np.float)
    return matrix

def split_target(y_train):
    matrix=np.array([0]*len(y_train),np.float)
    i=0
    #print matrix
    for train in y_train:
        #print train[0]
        if train=='H':
            matrix[i]=0
        elif train=='D':
            matrix[i]=1
        else:
            matrix[i]=2
        i+=1
    return matrix


def predict(data_loader,model):
    probas=[]
    for i,data in enumerate(data_loader,0):
        inputs,labels =data
        inputs,labels=Variable(inputs),Variable(labels)
        pred=model(inputs)
        probas.append(pred.data.numpy())
        
    return probas

def accuracy(data_loader,model):
    summe=0
    count=0
    for i,data in enumerate(data_loader,0):
        inputs,labels =data
        inputs,labels=Variable(inputs),Variable(labels)
        pred=model(inputs)
        #print pred
        _,predicted=torch.max(pred.data,1)
        predicted=predicted.numpy()
        labels=labels.data.numpy()
        for i in range(len(predicted)):
            if labels[i]==predicted[i]:
                summe+=1# labels[i][predicted[i]]
        
        count+=len(predicted)
    
    return float(summe)/float(count)

In [8]:

import torch
import torch.utils.data
from torch.autograd import Variable
import matplotlib.pyplot as plt
import glob

def decide_on_bet(probas,odds,result,thresh,league,tax,verbose,alpha,list_of_bets,list_of_data):
    exp_ret_home=probas[0]*odds[0]
    exp_ret_draw=probas[1]*odds[1]
    exp_ret_away=probas[2]*odds[2]
    bet=1
    win=0
    loss=0
    bet_count=0
    bet_amount=0
    if odds[0]==1.888:#2.3601
        pass
    else:
        #print exp_ret_home,exp_ret_draw,exp_ret_away
        if exp_ret_home>thresh and exp_ret_home > exp_ret_draw and exp_ret_home > exp_ret_away:
            if verbose:
                print 'In {} {} vs {} I bet {} euros on Home with odds {} and expected return {}'.format(odds[4],odds[5],odds[6],(exp_ret_home-thresh)*alpha,odds[0],exp_ret_home)
            bet_count+=1
            bet=((exp_ret_home-thresh)*alpha)
            bet_amount+=bet
            list_of_data.append([probas[0],odds[3],odds[4],bet,odds[5],odds[6],odds[7]])
            if result=='H':
                if verbose:
                    print 'and won {}'.format(bet*odds[0]*(1-tax)-bet)
                win+=bet*odds[0]*(1-tax)-bet
                list_of_bets.append(bet*odds[0]*(1-tax)-bet)
            else:
                if verbose:
                    print 'and lost {}'.format(-bet)
                loss+=bet
                list_of_bets.append(-bet)

        if exp_ret_draw>thresh and exp_ret_draw > exp_ret_home and exp_ret_draw > exp_ret_away:
            if verbose:
                print 'In {} {} vs {} I bet {} euros on Draw with odds {} and expected return {}'.format(odds[4],odds[5],odds[6],(exp_ret_draw-thresh)*alpha,odds[1],exp_ret_draw)
            bet_count+=1
            bet=((exp_ret_draw-thresh)*alpha)
            bet_amount+=bet
            list_of_data.append([probas[1],odds[3],odds[4],bet,odds[5],odds[6],odds[7]])
            if result=='D':
                if verbose:
                    print 'and won {}'.format(bet*odds[1]*(1-tax)-bet)
                win+=bet*odds[1]*(1-tax)-bet
                list_of_bets.append(bet*odds[1]*(1-tax)-bet)
            else:
                if verbose:
                    print 'and lost {}'.format(-bet)
                loss+=bet
                list_of_bets.append(-bet)

        if exp_ret_away>thresh and exp_ret_away > exp_ret_draw and exp_ret_away > exp_ret_home:
            if verbose:
                print 'In {} {} vs {} I bet {} euros on Away with odds {} and expected return {}'.format(odds[4],odds[5],odds[6],alpha*(exp_ret_away-thresh),odds[2],exp_ret_away)
            bet_count+=1
            bet=((exp_ret_away-thresh)*alpha)
            bet_amount+=bet
            list_of_data.append([probas[2],odds[3],odds[4],bet,odds[5],odds[6],odds[7]])
            if result=='A':
                if verbose:
                    print 'and won {}'.format(bet*odds[2]*(1-tax)-bet)
                win+=bet*odds[2]*(1-tax)-bet
                list_of_bets.append(bet*odds[2]*(1-tax)-bet)
            else:
                if verbose:
                    print 'and lost {}'.format(-bet)
                loss+=bet
                list_of_bets.append(-bet)
    return win-loss,bet_count,bet_amount,list_of_bets,list_of_data


def eval_model(model,val_loader,y_val):
    

    betting_data=y_val.drop(['FTR'],1).as_matrix()
    result_data=y_val['FTR'].as_matrix()

    threshs=[1]#np.linspace(0.5,2,10)
    alphas=[1]
    betting_tax=0
    verbose=False


    #best_model=torch.load('models/NN_63198_0.492167473654_0.473749169278.pt')

    probas=predict(val_loader,model)[0]
    #print probas[-100:]
    for thresh in threshs:
        #ret=[]
        for alpha in alphas:
            data_count=0
            won_overall=0
            bets_overall=0
            amount_overall=0
            list_of_bets=[]
            list_of_data=[]
            for i in range(len(betting_data)):
                won,num_bets,amount,list_of_bets,list_of_data=decide_on_bet(probas[i],betting_data[i],result_data[i],thresh,league,betting_tax,verbose,alpha,list_of_bets,list_of_data)
                amount_overall+=amount
                won_overall+=won
                bets_overall+=num_bets
    if amount_overall==0:
        return -10,-1
    else:
        return won_overall/float(amount_overall),bets_overall

    

In [ ]:
import torch
import torch.utils.data
from torch.autograd import Variable
import matplotlib.pyplot as plt
import glob
import numpy as np

def make_sparse(array):
    array=np.array(array.as_matrix(),dtype=np.float)
    #print array.shape
    v= torch.FloatTensor(array[~np.isnan(array)])
    #v= torch.FloatTensor([x for x in array if not np.isnan(x)])
    
    #print np.argwhere(np.invert(np.isnan(array)))
    i= torch.LongTensor(np.transpose(np.argwhere(np.invert(np.isnan(array)))))
    #print i
    #print v
    return torch.sparse.FloatTensor(i,v)
    
  


In [ ]:
def custom_loss(X,y,bet_data):
    could_have_won=bet_data.gather(1,y.view(-1,1))-1
    #print could_have_won
    bet= (torch.mul(X,bet_data)-1).clamp(0)
    #print bet
    won=bet.gather(1,y.view(-1,1))
    #print won
    loss= torch.sum(bet,dim=1).view(len(bet_data),1).clamp(0.05)#.sub_(won) 0.1
    #print loss
    won=torch.mul(won,bet_data.gather(1,y.view(-1,1)))
    #print won
    won=won-loss
    #print won
    loss=could_have_won-won
    #print loss
    loss=loss.clamp(0)
    loss=torch.sum(loss)
    could_have_won=torch.sum(could_have_won)
    loss= loss.div(could_have_won)
    return loss


In [ ]:
import torch
import torch.utils.data
from torch.autograd import Variable
import matplotlib.pyplot as plt
import copy

batch_sizes=[8000,9000,10000,12000]
input_dim=len(X_train.columns)
Hidden_dims=[int(input_dim/2),int(input_dim)]#,
output_dim=3

epochs=30

models=[]

#y_train.fillna(2,inplace=True)
X_val.fillna(0,inplace=True)
X_test.fillna(0,inplace=True)
#print torch.from_numpy(X_val.as_matrix()).float()
val_set=torch.utils.data.TensorDataset(torch.from_numpy(X_val.as_matrix()).float(),torch.from_numpy(split_target(y_val['FTR'])).long())
val_loader=torch.utils.data.DataLoader(val_set,batch_size=len(X_val),shuffle=False,num_workers=8)

test_set=torch.utils.data.TensorDataset(torch.from_numpy(X_test.as_matrix()).float(),torch.from_numpy(split_target(y_test['FTR'])).long())
test_loader=torch.utils.data.DataLoader(test_set,batch_size=len(X_test),shuffle=False,num_workers=8)

for Hidden_dim in Hidden_dims:
    
        
    models.append(torch.nn.Sequential(
        torch.nn.BatchNorm1d(input_dim),
        torch.nn.Linear(input_dim,Hidden_dim),
        torch.nn.Linear(Hidden_dim,2*Hidden_dim),
        torch.nn.Linear(2*Hidden_dim,3*Hidden_dim),
        torch.nn.Dropout(),
        torch.nn.Linear(Hidden_dim*3,2*Hidden_dim),
        torch.nn.Linear(2*Hidden_dim,Hidden_dim),
        torch.nn.ReLU(),
        torch.nn.Linear(Hidden_dim,Hidden_dim),
        torch.nn.Linear(Hidden_dim,output_dim),
        torch.nn.Sigmoid()))
    
    models.append(torch.nn.Sequential(
        torch.nn.BatchNorm1d(input_dim),
        torch.nn.Linear(input_dim,Hidden_dim),
        torch.nn.Linear(Hidden_dim,Hidden_dim),
        torch.nn.Linear(Hidden_dim,Hidden_dim),
        torch.nn.Dropout(),
        torch.nn.Linear(Hidden_dim,Hidden_dim),
        torch.nn.Linear(Hidden_dim,Hidden_dim),
        torch.nn.ReLU(),
        torch.nn.Linear(Hidden_dim,Hidden_dim),
        torch.nn.Linear(Hidden_dim,output_dim),
        torch.nn.Sigmoid()))
    
    models.append(torch.nn.Sequential(
        torch.nn.BatchNorm1d(input_dim),
        torch.nn.Linear(input_dim,Hidden_dim),
        torch.nn.Linear(Hidden_dim,Hidden_dim),
        torch.nn.Linear(Hidden_dim,Hidden_dim),
        torch.nn.Dropout(),
        torch.nn.Linear(Hidden_dim,2*Hidden_dim),
        torch.nn.Linear(2*Hidden_dim,3*Hidden_dim),
        torch.nn.ReLU(),
        torch.nn.Linear(3*Hidden_dim,Hidden_dim),
        torch.nn.Linear(Hidden_dim,output_dim),
        torch.nn.Sigmoid()))
    '''   
    models.append(torch.nn.Sequential(
        torch.nn.BatchNorm1d(input_dim),
        torch.nn.Linear(input_dim,Hidden_dim),
        torch.nn.Linear(Hidden_dim,2*Hidden_dim),
        torch.nn.Linear(2*Hidden_dim,3*Hidden_dim),
        torch.nn.Linear(Hidden_dim*3,2*Hidden_dim),
        torch.nn.Linear(2*Hidden_dim,Hidden_dim),
        torch.nn.Linear(Hidden_dim,Hidden_dim),
        torch.nn.Linear(Hidden_dim,output_dim),
        torch.nn.Sigmoid()))
    
    models.append(torch.nn.Sequential(
        torch.nn.BatchNorm1d(input_dim),
        torch.nn.Linear(input_dim,Hidden_dim),
        torch.nn.Linear(Hidden_dim,Hidden_dim),
        torch.nn.Linear(Hidden_dim,2*Hidden_dim),
        torch.nn.Dropout(p=0.2),
        torch.nn.Linear(Hidden_dim*2,Hidden_dim),
        torch.nn.Linear(Hidden_dim,Hidden_dim),
        torch.nn.ReLU(),
        torch.nn.Linear(Hidden_dim,Hidden_dim),
        torch.nn.Linear(Hidden_dim,output_dim),
        torch.nn.Sigmoid()))
    '''
'''    
b_m=torch.load("/home/angloher/projects/Predicting_Winning_Teams/models/current_set/best_model.pt")

pretrained_dict= b_m.state_dict()
for model in models:
    model_dict=model.state_dict()
    for i in range(len(pretrained_dict.items())):
        #print model_dict.items()[i]
        if len(model_dict.items()[i][1].shape)==2:
            print pretrained_dict.items()[i][1].shape[0]
            model_dict.items()[i][1][0:pretrained_dict.items()[i][1].shape[0],0:pretrained_dict.items()[i][1].shape[1]]=pretrained_dict.items()[i][1]
        else:
            model_dict.items()[i][1][0:len(pretrained_dict.items()[i][1])]=pretrained_dict.items()[i][1]
        #print model_dict.items()[i]
    model_dict.update(model_dict)
    model.load_state_dict(model_dict,strict=False)
#print pretrained_dict.items()[0]
'''
print len(models)
for batch_size in batch_sizes:
    #sparse_X_tensor=make_sparse(X_train)
    #print sparse_X_tensor.shape[0]
    #y_tensor=torch.from_numpy(split_train_target(y_train[['FTR','B365H','B365D','B365A']].as_matrix()))
    #print y_tensor.shape[0]
    train_set=torch.utils.data.TensorDataset(torch.from_numpy(X_train.as_matrix()).float(),torch.from_numpy(split_train_target(y_train[['FTR','B365H','B365D','B365A']].as_matrix())))
    train_loader=torch.utils.data.DataLoader(train_set,batch_size=batch_size,shuffle=True,num_workers=8)
    
    for model in models:
        #loss_fn=torch.nn.CrossEntropyLoss()
        learning_rates=[5e-3,1e-3,1e-4]
        for learning_rate in learning_rates:
            print model,learning_rate,batch_size
            #optimizer = torch.optim.SparseAdam(model.parameters(),lr=learning_rate)
            optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate,weight_decay=0.001)
            #scheduler=torch.optim.lr_scheduler.StepLR(optimizer,5)
            test_acc=[0]
            best_test_return=-1
            best_index=0
            train_acc=[0]
            time=[0]
            counter=0
            t=0
            while counter<epochs:
                total_loss=0
                #scheduler.step()
                for i,data in enumerate(train_loader):
                    #print data
                    #inputs=sparse_X_tensor
                    #labels=y_tensor
                    inputs,labels =data
                    bet_data= labels.numpy()
                    bet_data=[x[1:] for x in bet_data]
                    #bet_data=np.array(labels.numpy(),np.float())
                    labels=labels.gather(1,torch.zeros(len(labels),1).long()).long()
                    
                    
                    inputs,labels=Variable(inputs),Variable(labels)
                    model.train()
                    y_pred=model(inputs)
                    #print y_pred
                    #bet_data=y_train[i*batch_size:min((i+1)*batch_size,len(y_train))]
                    #bet_data.fillna(0,inplace=True)
                    #print bet_data
                    bet_data=Variable(torch.from_numpy(np.array(bet_data,np.float)).float(),requires_grad=True)
                    #print bet_data
                    loss=custom_loss(y_pred,labels,bet_data)

                    #loss=loss_fn(y_pred,labels)
                    #print loss
                    total_loss+=loss.data[0]

                    optimizer.zero_grad()

                    loss.backward()

                    optimizer.step()
                print (t,total_loss,counter)
                t+=1
                if (t%1)==0:
                    model.eval()
                    #train_acc.append(accuracy(train_loader,model))
                    current_acc=accuracy(test_loader,model)
                    current_return,bets_overall=eval_model(model,test_loader,y_test)
                    print current_acc,current_return,bets_overall/float(len(y_test))
                    counter+=1

                    if current_return>best_test_return:
                        best_test_return=current_return
                        best_model=copy.deepcopy(model)
                        best_t=t
                        best_index=len(test_acc)
                        counter=0
                    test_acc.append(current_acc)
                    time.append(t)

            best_model.eval()
            plt.plot(time,test_acc)
            #plt.plot(time,train_acc)
            plt.show()
            print best_test_return
            print 'best model at {} iterations with test accuracy: {}\nvalid accuracy: {} '.format(best_index,test_acc[best_index],accuracy(val_loader,best_model))

            torch.save(best_model, "/home/angloher/projects/Predicting_Winning_Teams/models/NN_"+str(best_test_return)+"_"+str(bets_overall/float(len(y_test)))+".pt")


In [ ]:

import torch
import torch.utils.data
from torch.autograd import Variable
import matplotlib.pyplot as plt
import glob,os
#prediction
val_set=torch.utils.data.TensorDataset(torch.from_numpy(X_train.as_matrix()).float(),torch.from_numpy(split_target(y_train['FTR'])).float())
val_loader=torch.utils.data.DataLoader(val_set,batch_size=len(X_train),shuffle=False,num_workers=8)

best_models=[]
for file in glob.glob("/home/angloher/projects/Predicting_Winning_Teams/models/current_set/best*.pt"):
    best_models.append(torch.load(file))#best_model2
print len(best_models)
#best_model3
betting_data=y_train.drop(['FTR'],1).as_matrix()
result_data=y_train['FTR'].as_matrix()


threshs=[1]#np.linspace(0.5,2,10)
alphas=[1]
betting_tax=0
verbose=False


probas=np.zeros((len(betting_data),3),np.float)    


for best_model in best_models:
    print best_model
    best_model.eval()
    probas+=predict(val_loader,best_model)[0]
    #print probas
    
probas/=float(len(best_models))
#print probas[-100:]
for thresh in threshs:
    #ret=[]
    for alpha in alphas:
        data_count=0
        won_overall=0
        bets_overall=0
        amount_overall=0
        list_of_bets=[]
        list_of_data=[]
        for i in range(len(betting_data)):
            won,num_bets,amount,list_of_bets,list_of_data=decide_on_bet(probas[i],betting_data[i],result_data[i],thresh,league,betting_tax,verbose,alpha,list_of_bets,list_of_data)
            amount_overall+=amount
            won_overall+=won
            bets_overall+=num_bets

            #ret.append(won_overall/float(amount_overall))
        print 'average return on bets with overall is {}'.format(won_overall/float(amount_overall))
        print 'won overall {} in {} bets with overall bet amount {}'.format(won_overall,bets_overall,amount_overall)
        print ''
        #plt.plot(alphas,ret,label=str(thresh))
    #plt.legend()
    #plt.show()
#print list_of_data,betting_data

1
Sequential(
  (0): BatchNorm1d(281, eps=1e-05, momentum=0.1, affine=True)
  (1): Linear(in_features=281, out_features=281)
  (2): Linear(in_features=281, out_features=562)
  (3): Linear(in_features=562, out_features=843)
  (4): Dropout(p=0.5)
  (5): Linear(in_features=843, out_features=562)
  (6): Linear(in_features=562, out_features=281)
  (7): ReLU()
  (8): Linear(in_features=281, out_features=281)
  (9): Linear(in_features=281, out_features=3)
  (10): Sigmoid()
)


/home/angloher/projects/Predicting_Winning_Teams/PredictEnv/local/lib/python2.7/site-packages/torch/serialization.py:316: SourceChangeWarning: source code of class 'torch.nn.modules.container.Sequential' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/angloher/projects/Predicting_Winning_Teams/PredictEnv/local/lib/python2.7/site-packages/torch/serialization.py:316: SourceChangeWarning: source code of class 'torch.nn.modules.batchnorm.BatchNorm1d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/angloher/projects/Predicting_Winning_Teams/PredictEnv/local/lib/python2.7/site-packages/torch/serialization.py:316: SourceC

In [ ]:
import matplotlib.mlab as mlab
import scipy.stats

num_bets_per_week=100

dist=np.array(list(list_of_bets)).astype(np.float).reshape(len(list_of_bets),1)
dist= dist/np.array(list_of_data)[:,3:4].astype(np.float)
mean=np.average(dist,weights=np.array(list_of_data)[:,3:4].astype(np.float))
std=math.sqrt(np.average((dist-mean)**2,weights=np.array(list_of_data)[:,3:4].astype(np.float)))
std=std/np.sqrt(num_bets_per_week)
print 'mean: {}, std: {}'.format(mean,std)
print 'doubling rate: {} weeks'.format(0.7/mean)
x=np.linspace(mean-3*std,mean+3*std,100)
plt.plot(x,mlab.normpdf(x,mean,std))
plt.show()
print 'probability of losing money: {}'.format(scipy.stats.norm(mean,std).cdf(0))
print 'probability of losing half the money: {}'.format(scipy.stats.norm(mean,std).cdf(-0.5))
print 'probability of losing all the money: {}'.format(scipy.stats.norm(mean,std).cdf(-1))

In [ ]:
dist=np.array(list(list_of_bets)).astype(np.float).reshape(len(list_of_bets),1)
#print dist
num_bets=60
for t in range(1000):
    won=0
    for i in range(num_bets):
        won+=dist[np.random.randint(0,len(dist))]
    print won

In [ ]:
import re

proba_auswertung=[[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
probas_dist=np.linspace(0,95,20)
high_certainty=[]

year_auswertung=[[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
year_dist=np.linspace(1993,2018,26)

runde_auswertung=[[],[]]

Spieltag_auswertung=[[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
Spieltag_dist=np.linspace(0,50,51)

div_auswertung=[[],[],[],[],[],[],[],[],[],[]]
Divisions=['B','D','E','F','G','I','N','P','S','T']

print len(year_auswertung)
print year_dist
#print list_of_data

for i in range(len(list_of_bets)):
    proba_auswertung[int(list_of_data[i][0]*100/5)-1].append(list_of_bets[i])
    if list_of_data[i][0]*100>99:
        high_certainty.append(list_of_bets[i])
    
    #print int(re.split('-',list_of_data[i][1])[0])-1993
    year_auswertung[int(re.split('-',list_of_data[i][1])[0])-1993].append(list_of_bets[i])
    if int(re.split('-',list_of_data[i][1])[1])==3:
        runde_auswertung[1].append(list_of_bets[i])
    else:
        runde_auswertung[0].append(list_of_bets[i])
    
    #print list_of_data[i]
    #print list_of_data[i][6]
    #print Spieltag_auswertung
    Spieltag_auswertung[list_of_data[i][6]].append(list_of_bets[i])
    
    for j in range(len(Divisions)):
        if list_of_data[i][2][0]==Divisions[j]:
            div_auswertung[j].append(list_of_bets[i])

for i in range(len(proba_auswertung)):
    proba_auswertung[i]=np.mean(proba_auswertung[i])
    
for i in range(len(year_auswertung)):
    year_auswertung[i]=np.mean(year_auswertung[i])

print Spieltag_auswertung[1]
for i in range(len(Spieltag_auswertung)):
    Spieltag_auswertung[i]=np.mean(Spieltag_auswertung[i])    
    
for i in range(len(div_auswertung)):
    div_auswertung[i]=np.mean(div_auswertung[i])

print np.mean(high_certainty)    
plt.plot(probas_dist,proba_auswertung)
plt.show()
plt.clf()

plt.plot(year_dist,year_auswertung)
plt.show()
plt.clf()

plt.plot(Divisions,div_auswertung)
plt.show()
plt.clf()

plt.plot(Spieltag_dist,Spieltag_auswertung)
plt.show()
plt.clf()

print np.mean(runde_auswertung[0]),np.mean(runde_auswertung[1])

In [ ]:
#print X_validate_new_result_set
val_set=torch.utils.data.TensorDataset(torch.from_numpy(X_validate_new_result_set.as_matrix()).float(),torch.from_numpy(split_target(y_validate_new_result_set['FTR'])).float())
val_loader=torch.utils.data.DataLoader(val_set,batch_size=len(X_validate_new_result_set),shuffle=False,num_workers=8)
#best_model=torch.load("/home/angloher/projects/Predicting_Winning_Teams/models/best_model.pt")

betting_data=y_validate_new_result_set.drop(['FTR'],1).as_matrix()
result_data=y_validate_new_result_set['FTR'].as_matrix()

threshs=[1]#np.linspace(0.5,2,10)
alphas=[1]
betting_tax=0
verbose=True

    
#best_model=torch.load('models/NN_63198_0.492167473654_0.473749169278.pt')

probas=np.zeros((len(betting_data),3),np.float)   
print betting_data
for best_model in best_models:
    best_model.eval()
    probas+=predict(val_loader,best_model)[0]
    #print probas
    
probas/=float(len(best_models))

print len(probas)
for thresh in threshs:
    #ret=[]
    for alpha in alphas:
        data_count=0
        won_overall=0
        bets_overall=0
        amount_overall=0
        list_of_bets=[]
        list_of_data=[]
        for i in range(len(betting_data)):
            won,num_bets,amount,list_of_bets,list_of_data=decide_on_bet(probas[i],betting_data[i],result_data[i],thresh,league,betting_tax,verbose,alpha,list_of_bets,list_of_data)
            amount_overall+=amount
            won_overall+=won
            bets_overall+=num_bets

            #ret.append(won_overall/float(amount_overall))
        print 'average return on bets with overall is {}'.format(won_overall/float(amount_overall))
        print 'won overall {} in {} bets with overall bet amount {}'.format(won_overall,bets_overall,amount_overall)
        print ''
        #plt.plot(alphas,ret,label=str(thresh))
    #plt.legend()
    #plt.show()
#print list_of_data,betting_data
betrate=bets_overall/float(len(probas))
print betrate

In [ ]:
import re
import datetime

weeks_auswertung=[]
weeks=[]
week_counter=[]
weeks_data=[]

stop_date=datetime.date.today()


#start_date=datetime.datetime.strptime(''.join(train_val_split_date),'%m%d%Y').date()
start_date=datetime.datetime.strptime('09-02-2018','%d-%m-%Y').date()
 
week_count=0
while start_date+week_count*datetime.timedelta(4)<stop_date:
    weeks.append(start_date+week_count*datetime.timedelta(4))
    weeks_auswertung.append([])
    week_counter.append([])
    weeks_data.append([])
    week_count+=1
    
for i in range(len(list_of_bets)):
    for j in range(len(weeks)):    
        if datetime.datetime.strptime(list_of_data[i][1],'%Y-%m-%d').date()>=weeks[j]and datetime.datetime.strptime(list_of_data[i][1],'%Y-%m-%d').date()<weeks[j+1]:
            weeks_auswertung[j].append(list_of_bets[i])
            weeks_data[j].append(list_of_data[i])

lost=0
won=0
summe=0
einsatz=0
count=0
w=0
for c in range(len(weeks_auswertung[w])):
    #print c,weeks_auswertung[0][c],weeks_data[0][c]
    
    if weeks_data[w][c][3]*4>0.8:
        count+=1
        einsatz+=weeks_data[w][c][3]
        if weeks_auswertung[w][c]*weeks_data[w][c][3]>0.5:
            print weeks_data[w][c],weeks_auswertung[w][c]
            summe+=weeks_auswertung[w][c]*weeks_data[w][c][3]
        if weeks_auswertung[w][c]>0:    
            won+=(1+weeks_auswertung[w][c])*weeks_data[w][c][3]

print count
print einsatz*4        
print won*4
print summe/won
print won/float(len(weeks_auswertung[0]))

for i in range(len(weeks_auswertung)):
    week_counter[i]=len(weeks_auswertung[i])
    weeks_auswertung[i]=np.mean(weeks_auswertung[i])
    if np.mean(weeks_auswertung[i])<0:
        lost+=1
        
print lost/float(len(weeks))
print weeks_auswertung
print week_counter
print weeks

fig=plt.figure(figsize=(20,20))
print np.mean(weeks_auswertung[:-1])
print np.std(weeks_auswertung[:-1])
ax=fig.add_subplot(111)
ax.plot(weeks,weeks_auswertung)

fig.savefig('test.png')


In [ ]:

num_bets_per_week=150*betrate

dist=np.array(list(list_of_bets)).astype(np.float).reshape(len(list_of_bets),1)
dist= dist/np.array(list_of_data)[:,3:4].astype(np.float)
mean=np.average(dist,weights=np.array(list_of_data)[:,3:4].astype(np.float))
std=math.sqrt(np.average((dist-mean)**2,weights=np.array(list_of_data)[:,3:4].astype(np.float)))
std=std/np.sqrt(num_bets_per_week)
print 'mean: {}, std: {}'.format(mean,std)
print 'doubling rate: {} weeks'.format(0.7/mean)
x=np.linspace(mean-3*std,mean+3*std,100)
plt.plot(x,mlab.normpdf(x,mean,std))
plt.show()
print 'probability of losing money: {}'.format(scipy.stats.norm(mean,std).cdf(0))
print 'probability of losing half the money: {}'.format(scipy.stats.norm(mean,std).cdf(-0.5))
print 'probability of losing all the money: {}'.format(scipy.stats.norm(mean,std).cdf(-1))

In [ ]:
#prediction
test_set=torch.utils.data.TensorDataset(torch.from_numpy(X_test.as_matrix()).float(),torch.from_numpy(split_target(y_test['FTR'])).float())
test_loader=torch.utils.data.DataLoader(test_set,batch_size=len(X_test),shuffle=False,num_workers=8)

betting_data=y_test.drop(['FTR'],1).as_matrix()
result_data=y_test['FTR'].as_matrix()

threshs=[1]#np.linspace(0.5,2,10)
alphas=[1]#np.linspace(-5,5,20)
betting_tax=0
verbose=False


probas=np.zeros((len(betting_data),3),np.float)   

for best_model in best_models:
    best_model.eval()
    probas+=predict(test_loader,best_model)[0]
    #print probas
    
probas/=float(len(best_models))

for thresh in threshs:
        #ret=[]
    for alpha in alphas:
        print thresh,alpha
        data_count=0
        won_overall=0
        bets_overall=0
        amount_overall=0
        list_of_bets=[]
        list_of_data=[]
        for i in range(len(betting_data)):
            won,num_bets,amount,list_of_bets,list_of_data=decide_on_bet(probas[i],betting_data[i],result_data[i],thresh,league,betting_tax,verbose,alpha,list_of_bets,list_of_data)
            amount_overall+=amount
            won_overall+=won
            bets_overall+=num_bets

            #ret.append(won_overall/float(amount_overall))
        print 'average return on bets with overall is {}'.format(won_overall/float(amount_overall))
        print 'won overall {} in {} bets'.format(won_overall,bets_overall)
        print ''
        print won_overall
        print sum(list_of_bets)
        print amount_overall
        print sum(np.array(list_of_data)[:,3:4].astype(np.float))
       # plt.plot(alphas,ret,label=str(thresh))
    #plt.legend()
    #plt.show()

In [ ]:
import matplotlib.mlab as mlab
import scipy.stats

num_bets_per_week=50
#print np.array(list_of_data)[:,3:4]
dist=np.array(list(list_of_bets)).astype(np.float).reshape(len(list_of_bets),1)
dist= dist/np.array(list_of_data)[:,3:4].astype(np.float)
mean=np.average(dist,weights=np.array(list_of_data)[:,3:4].astype(np.float))
std=math.sqrt(np.average((dist-mean)**2,weights=np.array(list_of_data)[:,3:4].astype(np.float)))
std=std/np.sqrt(num_bets_per_week)
print 'mean: {}, std: {}'.format(mean,std)
print 'doubling rate: {} weeks'.format(0.7/mean)
x=np.linspace(mean-3*std,mean+3*std,100)
plt.plot(x,mlab.normpdf(x,mean,std))
plt.show()
print 'probability of losing money: {}'.format(scipy.stats.norm(mean,std).cdf(0))
print 'probability of losing half the money: {}'.format(scipy.stats.norm(mean,std).cdf(-0.5))
print 'probability of losing all the money: {}'.format(scipy.stats.norm(mean,std).cdf(-1))



In [ ]:
mean=0.087
std=0.13
prob_of_loss=0.01
num_weeks=20
percentage_of_money_bet=1
money=1
counter=0
avg_ret=0
testlength=1000
for j in range(testlength):
    money=1
    i=0
    while i <num_weeks:
        i+=1
        money+= percentage_of_money_bet*money*scipy.stats.norm.rvs(mean,std)
        #print i,money
    avg_ret+=money-1
    if money<1:
        counter+=1
print avg_ret/float(testlength)
print counter/float(testlength)



In [ ]:
best_models=[]
for file in glob.glob("/home/angloher/projects/Predicting_Winning_Teams/models/models*/NN_119934_*.pt"):
    best_models.append(torch.load(file))
    
test_set=torch.utils.data.TensorDataset(torch.from_numpy(X_test.as_matrix()).float(),torch.from_numpy(split_target(y_test['FTR'])).float())
test_loader=torch.utils.data.DataLoader(test_set,batch_size=len(X_test),shuffle=False,num_workers=8)

betting_data=y_test.drop(['FTR'],1).as_matrix()
result_data=y_test['FTR'].as_matrix()

threshs=[1]#np.linspace(0.5,2,10)
alphas=[1]#np.linspace(-5,5,20)
betting_tax=0
verbose=False


probas=np.zeros((len(betting_data),3),np.float)   

for best_model in best_models:
    probas=predict(test_loader,best_model)[0]
    print best_model

    for thresh in threshs:
            #ret=[]
        for alpha in alphas:
            print thresh,alpha
            data_count=0
            won_overall=0
            bets_overall=0
            amount_overall=0
            list_of_bets=[]
            list_of_data=[]
            for i in range(len(betting_data)):
                won,num_bets,amount,list_of_bets,list_of_data=decide_on_bet(probas[i],betting_data[i],result_data[i],thresh,league,betting_tax,verbose,alpha,list_of_bets,list_of_data)
                amount_overall+=amount
                won_overall+=won
                bets_overall+=num_bets

                #ret.append(won_overall/float(amount_overall))
            print 'average return on bets with overall is {}'.format(won_overall/float(amount_overall))
            print 'won overall {} in {} bets'.format(won_overall,bets_overall)
            print ''
            print won_overall
            print sum(list_of_bets)
            print amount_overall
            print sum(np.array(list_of_data)[:,3:4].astype(np.float))
           # plt.plot(alphas,ret,label=str(thresh))
        #plt.legend()
        #plt.show()